In [2]:
import numpy as np

# Define a Markov Decision Process for a quiz game
class QuizMDP:
    def __init__(self, rewards, probabilities):
        # Rewards for each question
        self.q_rewards = rewards
        # Probabilities of answering a question correctly
        self.q_probs = probabilities
        # Total number of questions
        self.num_questions = len(rewards)
        # Total number of states (questions + end state)
        self.num_states = self.num_questions + 1
        # Starting state
        self.start_state = 0
        # End state
        self.end_state = self.num_states - 1

    # Get the probabilities of transitioning to the next state
    def get_transition_probs(self, state, action):
        if state == self.end_state:
            # If already in the end state, stay there with probability 1
            return [(self.end_state, 1.0)]
        elif action == 1:
            # If answering a question, transition to the next state with probability q_probs[state]
            # or go to the end state with probability 1 - q_probs[state]
            return [(state + 1, self.q_probs[state]), (self.end_state, 1 - self.q_probs[state])]
        else:
            # If passing, go directly to the end state with probability 1
            return [(self.end_state, 1.0)]

    # Get the rewards for transitioning to the next state
    def get_rewards(self, state, action, next_state):
        if next_state == self.end_state:
            # If already in the end state, get no reward
            return 0
        elif action == 1:
            # If answering a question, get the reward for the current state
            return self.q_rewards[state]
        else:
            # If passing, get no reward
            return 0

# Use value iteration to find the optimal policy
def find_optimal_policy(quiz, gamma=1.0, theta=1e-6):
    # Initialize the value function to zero
    V = np.zeros(quiz.num_states)

    while True:
        # Initialize the change in V to zero
        delta = 0

        # Update the value function for each state
        for s in range(quiz.num_states):
            v = V[s]
            # Get the maximum expected value for each action
            V[s] = max([sum([p * (quiz.get_rewards(s, a, next_s) + gamma * V[next_s])
                             for (next_s, p) in quiz.get_transition_probs(s, a)])
                        for a in range(2)])
            # Update delta to keep track of the largest change in V
            delta = max(delta, abs(v - V[s]))

        # Check for convergence
        if delta < theta:
            break

    return V